In [1]:
#Extract properties from watershed
import glob
import h5py
from skimage.measure import label, regionprops
from matplotlib import pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix,lil_matrix
import cv2
import os
import tensorly as tl

In [2]:
#Sort .h5 and .tif datasets
def custom_sorth5(name_fov): 
    [other,value]=name_fov.split('sub')
    [value,other,other]=value.split('_')    
    return value
def custom_sortdapi(name_dapi): 
    [other,value]=name_dapi.split('sub')
    [value,other]=value.split('.')    
    return value

In [4]:
#Read all h5 files in a directory. 
h5dir = '/home/garner1/Work/dataset/tissue2graph/h5/'
tifdir = '/home/garner1/Work/dataset/tissue2graph/tif/'

In [5]:
h5_files = []
for file in os.listdir(h5dir):
    if file.endswith(".h5"):
        h5_files.append(os.path.join(datadir, file))
        
dapi_files = []
for file in os.listdir(tifdir):
    if file.endswith(".tif"):
        dapi_files.append(os.path.join(datadir, file))

In [6]:
#Read all tif files in directory
dapi_files.sort(key=custom_sortdapi)
h5_files.sort(key=custom_sorth5)

In [7]:
#Get all the FOVs and arrange them in the position
#To get the range we have to add one
number_fovs=np.arange(1,len(h5_files)+1) #these are the labels of the fov
fov_matrix=np.array(number_fovs).reshape(23,25) #reshape according to the patches 2D structure

In [88]:
dapi_fov_counter=0 #Set counter to check current fov
sintensity=[] #Create a list in which to save the sparses
row_list = [] #list of rows coordinates of the fov
col_list = [] #list of cols coordinates of the fov
centroids = []

for current_fov in h5_files[:2]:
    print(str(dapi_fov_counter+1)+' of '+str(len(h5_files)))
    fov = h5py.File(current_fov, 'r')
    
    mask=fov['/exported_watershed_masks'][:]
    mask_reduced=np.squeeze(mask, axis=2)
    
    dapi_fov= cv2.imread(dapi_files[dapi_fov_counter],cv2.IMREAD_GRAYSCALE) #Get DAPI fov 

    #Check which position the FOV occupies within the big scan
    #Position of FOV ilastik_masks_watershed1.h5
    [other,value]=current_fov.split('sub')
    [value,other,other]=value.split('_')
    (x,y)=np.where(fov_matrix==int(value))
    mask_label=label(mask_reduced) # label all connected components in the fov, 0 is background

    row_list.append(x)
    col_list.append(y)
    
    coords = [] #list of centroid coordinates for sc in each fov
    for region in regionprops(mask_label):
            centroid = region.centroid
            coords.append((int(centroid[0]),int(centroid[1])))
    centroids.append(coords)            
#     print(x,y)
#     plt.imshow(mask_reduced)
#     plt.show()
    
    type(mask)
    [height,width]=mask_reduced.shape

    #Create a 3D sparse array where x,y are FOV size and z is the amount of nuclei in the FOV 
    sparse_in_fov=[]
    z_dimension=0 #Set a counter to check current stack
    for i in range(1,np.amax(mask_label)+1): # 0 is background so it doesn't get included in the range
        xmask,ymask=np.where(mask_label==i)
        single_cell_mask=lil_matrix((1024, 1024), dtype='uint8')
        single_cell_mask[xmask,ymask]=dapi_fov[xmask,ymask]
        sparse_in_fov.append(single_cell_mask) #Add current nuclei sparse on to the FOV array
        z_dimension+=1 #Move to the next stack (next nuclei label)

    sintensity.append(sparse_in_fov) #Save all sparse array in the fov
    dapi_fov_counter+=1 #Move to the next dapi fov
    
#Save properties
np.savez('fov_data.npz',sintensity=sintensity,row_list=row_list,col_list=col_list,centroids=centroids)

1 of 575
2 of 575


In [78]:
'''
fov_id = 1; cell_id = 1
obj[fov_id][0] # row
obj[fov_id][1] # col
obj[fov_id][2][cell_id] # centroids
obj[fov_id][3][cell_id] # sparse matrix of intensities
'''
obj = list(zip(row_list,col_list,fov_centroids,all_fov_sparses)) # obj[ind][]

ValueError: Object arrays cannot be loaded when allow_pickle=False